# Sathvik Radhakrishnan

## Research question/interests

I would like to know the relationships between different parameters (nationality, age, position, speed) and their relationship to a players market value.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/raw/FIFA22_official_data.csv")

df.head(10)

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,212198,Bruno Fernandes,26,https://cdn.sofifa.com/players/212/198/22_60.png,Portugal,https://cdn.sofifa.com/flags/pt.png,88,89,Manchester United,https://cdn.sofifa.com/teams/11/30.png,...,65.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,€206.9M,72.0
1,209658,L. Goretzka,26,https://cdn.sofifa.com/players/209/658/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.com/teams/21/30.png,...,77.0,13.0,8.0,15.0,11.0,9.0,CM,87.0,€160.4M,74.0
2,176580,L. Suárez,34,https://cdn.sofifa.com/players/176/580/22_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,88,88,Atlético de Madrid,https://cdn.sofifa.com/teams/240/30.png,...,38.0,27.0,25.0,31.0,33.0,37.0,ST,88.0,€91.2M,42.0
3,192985,K. De Bruyne,30,https://cdn.sofifa.com/players/192/985/22_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,91,91,Manchester City,https://cdn.sofifa.com/teams/10/30.png,...,53.0,15.0,13.0,5.0,10.0,13.0,CM,91.0,€232.2M,68.0
4,224334,M. Acuña,29,https://cdn.sofifa.com/players/224/334/22_60.png,Argentina,https://cdn.sofifa.com/flags/ar.png,84,84,Sevilla FC,https://cdn.sofifa.com/teams/481/30.png,...,82.0,8.0,14.0,13.0,13.0,14.0,LB,84.0,€77.7M,80.0
5,212622,J. Kimmich,26,https://cdn.sofifa.com/players/212/622/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,89,90,FC Bayern München,https://cdn.sofifa.com/teams/21/30.png,...,79.0,8.0,15.0,7.0,15.0,15.0,CDM,89.0,€186.3M,85.0
6,187961,Paulinho,32,https://cdn.sofifa.com/players/187/961/22_60.png,Brazil,https://cdn.sofifa.com/flags/br.png,83,83,Al Ahli,https://cdn.sofifa.com/teams/112387/30.png,...,75.0,16.0,16.0,11.0,6.0,8.0,CM,83.0,€48.5M,80.0
7,181291,G. Wijnaldum,30,https://cdn.sofifa.com/players/181/291/22_60.png,Netherlands,https://cdn.sofifa.com/flags/nl.png,84,84,Paris Saint-Germain,https://cdn.sofifa.com/teams/73/30.png,...,70.0,16.0,6.0,14.0,12.0,6.0,CM,84.0,€74.9M,79.0
8,208333,E. Can,27,https://cdn.sofifa.com/players/208/333/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,82,83,Borussia Dortmund,https://cdn.sofifa.com/teams/22/30.png,...,86.0,14.0,8.0,8.0,13.0,11.0,CDM,85.0,€62.1M,81.0
9,194765,A. Griezmann,30,https://cdn.sofifa.com/players/194/765/22_60.png,France,https://cdn.sofifa.com/flags/fr.png,85,85,Atlético de Madrid,https://cdn.sofifa.com/teams/240/30.png,...,49.0,14.0,8.0,14.0,13.0,14.0,CAM,86.0,NaN,43.0
